In [1]:
import os
import json
import re

In [2]:
def merge_details(details):
    merged_list = []
    current_item = ""

    for i, item in enumerate(details):
        if item.strip() and item.strip()[0].isdigit():
            j = 1
            while j < len(item.strip()) and item.strip()[j].isdigit():
                j += 1
            if j < len(item.strip()) and item.strip()[j] == "：":
                merged_list.append(current_item)
                current_item = item.strip()
            else:
                current_item += item.strip()
        else:
            current_item += item.strip()

    merged_list.append(current_item)
    merged_list.pop(0)
    return merged_list


def extract_tuples(string):
    tuples = []
    depth = 0
    start = None
    for i, char in enumerate(string):
        if char == "(" or char == "（":
            if depth == 0:
                start = i
            depth += 1
        elif char == ")" or char == "）":
            depth -= 1
            if depth == 0 and start is not None:
                tuples.append(string[start+1:i])
                start = None
    return tuples


def get_results(merged_list, input_list, eve_extend_map, mode):
    eve_extend_index = 0
    cur_eve_extend_list = []
    A = []

    for idx, item in enumerate(merged_list):
        B = []
        cur_output_3 = item.split('Output:')
        cur_idx = cur_output_3[0].split('：')[0]
        cur_true_eve = input_list[int(cur_idx)-1]['event_type']
            
        if eve_extend_index == len(cur_eve_extend_list):
            eve_extend_index = 0

        cur_eve_extend_list = eve_extend_map[cur_true_eve]
        cur_extend_eve = cur_eve_extend_list[eve_extend_index]


        for output in cur_output_3[1:]:
            sep = ""
            if 'zh' in mode:
                if "生成事件检测列表:" in output:
                    sep = "生成事件检测列表:"
            elif 'en' in mode:
                if "Generate a list of event detections:" in output:
                    sep = "Generate a list of event detections:"
                elif "Generated event detection list:" in output:
                    sep = "Generated event detection list:"
                elif "Event detection list:" in output:
                    sep = "Event detection list:"
                elif "Event detections:" in output:
                    sep = "Event detections:"
            if sep:
                tmp = output.split(sep)[1]
                matches = extract_tuples(tmp)
                if matches:
                    for match in matches:
                        if 'zh' in mode:
                            triple = tuple(match.strip().strip('（）').split(','))
                        elif 'en' in mode:
                            triple = tuple(match.strip().strip('()').split(','))
                        if len(triple) == 2:
                            B.append((int(cur_idx)-1, cur_extend_eve, triple[1].strip()))
                        else:
                            B.append((int(cur_idx)-1, cur_extend_eve, ''))
                else:
                    B.append((int(cur_idx)-1, cur_extend_eve, ''))
                
            else:
                B.append((int(cur_idx)-1, cur_extend_eve, ''))
                continue
        if B:
            A.append(B)    

        eve_extend_index += 1

    A = [list(set(item)) for item in A]
    
    with open(f'./results/test_{mode}.txt','w',encoding='utf-8') as f:
        for item in A:
            f.write(str(item)+'\n')

DuEE

In [16]:
# 处理反向验证的输出文件
with open('./results/details_zh.txt','r',encoding='utf-8') as f:
    test_details = f.readlines()

with open('./data/duee/input_list.json','r',encoding='utf-8') as f:
    input_list = json.load(f)

with open('./data/duee/event_extend_map.json','r',encoding='utf-8') as f:
    eve_extend_map = json.load(f)

merged_list = merge_details(test_details)
get_results(merged_list, input_list, eve_extend_map, 'zh')


CASIE

In [18]:
# 处理反向验证的输出文件
with open('./results/details_en.txt','r',encoding='utf-8') as f:
    test_details = f.readlines()

with open('./data/casie/input_list.json','r',encoding='utf-8') as f:
    input_list = json.load(f)

with open('./data/casie/event_extend_map.json','r',encoding='utf-8') as f:
    eve_extend_map = json.load(f)

merged_list = merge_details(test_details)
get_results(merged_list, input_list, eve_extend_map, 'en')


ChatGLM-DuEE

In [6]:
# 处理反向验证的输出文件
with open('./results/details_chatglm_6B_zh.txt','r',encoding='utf-8') as f:
    test_details = f.readlines()

with open('./data/duee/input_list.json','r',encoding='utf-8') as f:
    input_list = json.load(f)

with open('./data/duee/final_event_extend_map_chatglm_6B_zh.json','r',encoding='utf-8') as f:
    eve_extend_map = json.load(f)

merged_list = merge_details(test_details)
get_results(merged_list, input_list, eve_extend_map, 'chatglm_6B_zh')


ChatGLM-CASIE

In [7]:
# 处理反向验证的输出文件
with open('./results/details_chatglm_6B_en.txt','r',encoding='utf-8') as f:
    test_details = f.readlines()

with open('./data/casie/input_list.json','r',encoding='utf-8') as f:
    input_list = json.load(f)

with open('./data/casie/final_event_extend_map_chatglm_6B_en.json','r',encoding='utf-8') as f:
    eve_extend_map = json.load(f)

merged_list = merge_details(test_details)
get_results(merged_list, input_list, eve_extend_map, 'chatglm_6B_en')


Baichuan2-DuEE

In [3]:
# 处理反向验证的输出文件
with open('./results/details_baichuan2_13B_zh.txt','r',encoding='utf-8') as f:
    test_details = f.readlines()

with open('./data/duee/input_list.json','r',encoding='utf-8') as f:
    input_list = json.load(f)

with open('./data/duee/final_event_extend_map_baichuan2_13B_zh.json','r',encoding='utf-8') as f:
    eve_extend_map = json.load(f)

merged_list = merge_details(test_details)
get_results(merged_list, input_list, eve_extend_map, 'baichuan2_13B_zh')


Baichuan2-CASIE

In [4]:
# 处理反向验证的输出文件
with open('./results/details_baichuan2_13B_en.txt','r',encoding='utf-8') as f:
    test_details = f.readlines()

with open('./data/casie/input_list.json','r',encoding='utf-8') as f:
    input_list = json.load(f)

with open('./data/casie/final_event_extend_map_baichuan2_13B_en.json','r',encoding='utf-8') as f:
    eve_extend_map = json.load(f)

merged_list = merge_details(test_details)
get_results(merged_list, input_list, eve_extend_map, 'baichuan2_13B_en')


Alpaca

In [4]:
# 处理反向验证的输出文件
with open('./results/details_alpaca_33B_zh.txt','r',encoding='utf-8') as f:
    test_details = f.readlines()

with open('./data/duee/input_list.json','r',encoding='utf-8') as f:
    input_list = json.load(f)

with open('./data/duee/final_event_extend_map_alpaca_33B_zh.json','r',encoding='utf-8') as f:
    eve_extend_map = json.load(f)

merged_list = merge_details(test_details)
get_results(merged_list, input_list, eve_extend_map, 'alpaca_33B_zh')


In [5]:
# 处理反向验证的输出文件
with open('./results/details_alpaca_33B_en.txt','r',encoding='utf-8') as f:
    test_details = f.readlines()

with open('./data/casie/input_list.json','r',encoding='utf-8') as f:
    input_list = json.load(f)

with open('./data/casie/final_event_extend_map_alpaca_33B_en.json','r',encoding='utf-8') as f:
    eve_extend_map = json.load(f)

merged_list = merge_details(test_details)
get_results(merged_list, input_list, eve_extend_map, 'alpaca_33B_en')
